In [6]:
from z3 import *
fml = """(declare-const a Bool)
(declare-const b Bool)
(declare-const c Bool)
(declare-const d Bool)
(declare-const e Bool)
(declare-const f Bool)
(declare-const g Bool)
(declare-const h Bool)
(declare-const i Bool)
(assert 
(and (or (not e) (not f) (not h))
          (or (not f) (not h) d)
          (or (not e) (not h) c)
          (or (not h) d c)
          (or (not e) (not f) (not g) a)
          (or (not f) d (not g) a)
          (or (not e) (not f) a i)
          (or (not f) d a i)
          (or (not e) c (not g) a)
          (or d c (not g) a)
          (or (not e) c a i)
          (or d c a i)
          (or (not e) (not f) a b)
          (or (not f) d a b)
          (or (not e) c a b)
          (or d c a b)))"""

s1 = Solver()
s2 = Solver()
s1.from_string(fml)
print(s1)
s2.add(Not(And(s1.assertions())))
s2.set("core.minimize", True)

clauses = []
while sat == s1.check():
    mdl = s1.model()
    decls = mdl.decls()
    core = [d() for d in decls if is_true(mdl[d])] + [Not(d()) for d in decls if is_false(mdl[d])] 
    assert unsat == s2.check(core)
    T = s2.unsat_core()
    clause = Or([mk_not(c) for c in T])
    print(core, T, clause)
    clauses += [clause]
    s1.add(clause)


for c in clauses:
    print(c)

[And(Or(Not(e), Not(f), Not(h)),
     Or(Not(f), Not(h), d),
     Or(Not(e), Not(h), c),
     Or(Not(h), d, c),
     Or(Not(e), Not(f), Not(g), a),
     Or(Not(f), d, Not(g), a),
     Or(Not(e), Not(f), a, i),
     Or(Not(f), d, a, i),
     Or(Not(e), c, Not(g), a),
     Or(d, c, Not(g), a),
     Or(Not(e), c, a, i),
     Or(d, c, a, i),
     Or(Not(e), Not(f), a, b),
     Or(Not(f), d, a, b),
     Or(Not(e), c, a, b),
     Or(d, c, a, b))]
[d, Not(i), Not(h), Not(g), Not(b), Not(f), Not(e), Not(c), Not(a)] [d, Not(e)] Or(Not(d), e)
[c, Not(f), Not(d)] [c, Not(f)] Or(Not(c), f)
[i, b, Not(h), Not(e), Not(c), Not(d), Not(g)] [b, i, Not(g), Not(h)] Or(Not(b), Not(i), g, h)
[i, a, Not(h), Not(c), Not(d), Not(b)] [a, Not(h)] Or(Not(a), h)
Or(Not(d), e)
Or(Not(c), f)
Or(Not(b), Not(i), g, h)
Or(Not(a), h)


In [2]:
def convert2cnf(exp):
    s1 = Solver()
    s2 = Solver()
    s1.add(Not(exp))
    s2.add(Not(And(s1.assertions())))
    s2.set("core.minimize", True)

    clauses = []
    while sat == s1.check():
        mdl = s1.model()
        decls = mdl.decls()
        core = [d() for d in decls if is_true(mdl[d])] + [Not(d()) for d in decls if is_false(mdl[d])] 
        assert unsat == s2.check(core)
        T = s2.unsat_core()
        clause = Or([mk_not(c) for c in T])
        print(core, T, clause)
        clauses += [clause]
        s1.add(clause)
    return clauses

In [3]:
def convert2dnf(exp):
    pass

In [4]:
a,b,c = Bools('a b c')
x = Or(And(Not(a),Not(b)), And(b,c), And(a,Not(c)))

In [5]:
convert2cnf(x)

[b, Not(a), Not(c)] [Not(a), Not(c), b] Or(a, c, Not(b))
[a, c, Not(b)] [Not(b), a, c] Or(b, Not(a), Not(c))


[Or(a, c, Not(b)), Or(b, Not(a), Not(c))]